# 高エネルギー実験で生成される荷電粒子の飛跡を、変分量子回路を使って再構成する

この実習では、変分量子回路を用いた**変分量子固有値ソルバー法**（*Variational Quantum Eigensolver*、VQE）を理解し、その基本的な利用方法を学びます。
また、高エネルギー実験では必須の技術である**「荷電粒子飛跡の再構成」**を、VQEを使って実現する初期的な実装を紹介します。

## 内容
1. [はじめに](#introduction)
2. [量子力学における変分法](#varmethod)
    1. [数学的背景](#backgroundmath)
    2. [基底状態の近似](#groundstate)
3. [変分量子固有値ソルバー法](#vqe)
    1. [変分量子回路](#varforms)
    2. [単純な変分フォーム](#simplevarform)
    3. [パラメータの最適化](#optimization)
    4. [変分フォームを使った実例](#example)
4. [高エネルギー実験への応用](#implementation)
    1. [量子演算の定義](#operators)
    2. [QUBO問題の導入](#qubo)
    3. [VQEによる近似解の探索](#vqe_tracking)
    4. [おまけ](#omake)
5. [参考文献](#references)

## はじめに<a id='introduction'></a>
行列で表現されるある物理系に対して、その固有値の最小値を見つけるという操作は、多くのアプリケーションで必要となる重要な技術です。例えば化学では、分子を特徴づけるエルミート行列の最小固有値は、そのシステムの基底状態のエネルギーになります。最小固有値を見つけるには**「量子位相推定アルゴリズム」**[1]を使うことができますが、実用的な応用問題の実装に必要な回路は、NISQ量子コンピュータでは実現できないほど長くなることが知られています。そのために、浅い量子回路を利用して分子の基底状態エネルギーを推定する手法として、**VQE**が提案されました[[2]](https://www.nature.com/articles/ncomms5213)。

まず、VQEの元になる関係を形式的に表現してみましょう。何か分からない最小固有値$\lambda_{min}$とその固有状態$|\psi_{min}\rangle$をもったエルミート行列$H$が与えられたとして、VQEはエネルギーの下限である$\lambda_{min}$の近似解$\lambda_{\theta}$を求める手法です。つまり

\begin{align*}
    \lambda_{min} \le \lambda_{\theta} \equiv \langle \psi(\theta) |H|\psi(\theta) \rangle
\end{align*}  

を満たす、できるだけ小さい$\lambda_{\theta}$を求めることに対応します。ここで$|\psi(\theta)\rangle$は近似解$\lambda_{\theta}$に対応する固有状態で、$\theta$はパラメータです。つまり、このアルゴリズムでは適当な初期状態$|\psi\rangle$に$U(\theta)$で表現されるパラメータ化された回路を適用することで、$|\psi_{min}\rangle$を近似する状態$U(\theta)|\psi\rangle \equiv |\psi(\theta)\rangle$が得られます。最適なパラメータ$\theta$の値は、期待値 $\langle \psi(\theta) |H|\psi(\theta) \rangle$が最小になるように古典計算を繰り返しながら求めていくことになります。


## 量子力学における変分法<a id='varmethod'></a>
### 数学的背景<a id='backgroundmath'></a>

VQEは量子力学の**変分法**を応用した手法です。変分法をよりよく理解するために、基礎的な数学的背景を説明します。行列$A$の固有ベクトル$|\psi_i\rangle$とその固有値$\lambda_i$は

\begin{align*}
    A |\psi_i\rangle = \lambda_i |\psi_i\rangle
\end{align*}

の関係にあります。行列$H$がエルミート行列$H = H^{\dagger}$の場合、スペクトル定理から$H$の固有値は実数になります（$\lambda_i = \lambda_i^*$）。測定できる量は実数である必要があるため、量子系のハミルトニアンを記述するためにはエルミート行列が適切です。さらに、$H$は以下のように表現できます。

\begin{align*}
    H = \sum_{i = 1}^{N} \lambda_i |\psi_i\rangle \langle \psi_i |
\end{align*}

ここで、各$\lambda_i$は対応する固有ベクトル$|\psi_i\rangle$の固有値です。任意の量子状態に対して観測量$H$を測定した場合の期待値は、以下の式で求められます。

\begin{align}
    \langle H \rangle_{\psi} &\equiv \langle \psi | H | \psi \rangle
\end{align}

上式の$H$を期待値の式に代入すると

\begin{align}
    \langle H \rangle_{\psi} = \langle \psi | H | \psi \rangle &= \langle \psi | \left(\sum_{i = 1}^{N} \lambda_i |\psi_i\rangle \langle \psi_i |\right) |\psi\rangle\\
    &= \sum_{i = 1}^{N} \lambda_i \langle \psi | \psi_i\rangle \langle \psi_i | \psi\rangle \\
    &= \sum_{i = 1}^{N} \lambda_i | \langle \psi_i | \psi\rangle |^2
\end{align}

になります。最後の式は、任意の状態$|\psi\rangle$に対する$H$の期待値は、$\lambda_i$を重みとした固有ベクトル$|\psi_i\rangle$と$|\psi\rangle$の内積の線形結合として与えられることを示しています。この式から、$| \langle \psi_i | \psi\rangle |^2 \ge 0$ であるために

\begin{align}
    \lambda_{min} \le \langle H \rangle_{\psi} = \langle \psi | H | \psi \rangle = \sum_{i = 1}^{N} \lambda_i | \langle \psi_i | \psi\rangle |^2
\end{align}

が成り立つことは明らかです。上記の式が**変分法**と呼ばれるもの（テキストによっては**変分原理**と呼ぶ）[3]で、$H$の最小固有値を下限として、任意の波動関数の期待値を近似的に求めることができることを表しています。この式から、$|\psi_{min}\rangle$状態の期待値は$\langle \psi_{min}|H|\psi_{min}\rangle = \langle \psi_{min}|\lambda_{min}|\psi_{min}\rangle = \lambda_{min}$になることも分かるでしょう。


### 基底状態の近似<a id='groundstate'></a>
系のハミルトニアンがエルミート行列$H$で表現されている場合、系の基底状態のエネルギーは$H$の最小固有値になります。まず$|\psi_{min}\rangle$の初期推定として任意の波動関数$|\psi \rangle$（*Ansatz*と呼ばれる）を選び、その状態での期待値$\langle H \rangle_{\psi}$を計算します。変分法の鍵は、この期待値が小さくなるように波動関数を更新しながら計算を繰り返し、ハミルトニアンの基底状態エネルギーに近づけていくことです。

## 変分量子固有値ソルバー法<a id='vqe'></a>
### 変分量子回路<a id='varforms'></a>
量子コンピューター上で変分法を実装するには、系統的に*Ansatz*を更新する方法が必要です。VQEはこれを決まった構造を持つパラメータ化された量子回路（**変分量子回路**）を使って行います。この回路はしばしば**変分フォーム**（*variational form*）と呼ばれ、ユニタリー変換$U(\theta)$で表現されます（$\theta$はパラメータ）。変分フォームを初期状態$|\psi\rangle$（例えば標準状態$|0\rangle$）に適用すると、出力として$U(\theta)|\psi\rangle\equiv |\psi(\theta)\rangle$が生成されます。この状態の元で、期待値$\langle \psi(\theta)|H|\psi(\theta)\rangle$が$\lambda_{min}$に近付くように、$|\psi(\theta)\rangle$に対してパラメータ$\theta$の最適化を行うことになります。

変分フォームの決め方ですが、解きたい問題のドメインに応じて特定の構造を持つ変分フォームを導入することがあります。そうではなく、幅広い問題への応用ができるようにドメインに依存しない形の変分フォーム（例えば$R_Y$ゲート）を使うこともあります。高エネルギー実験への応用では、この$R_Y$ゲートを使った変分フォームを実装します。

### 単純な変分フォーム<a id='simplevarform'></a>
変分フォームを決める時には、2つの相反する目的に対してバランスを考える必要があります。$n$量子ビットの変分フォームは、パラメータの数を増やせば$|\psi\rangle \in \mathbb{C}^N$かつ$N=2^n$の任意の状態$|\psi\rangle$を生成できるでしょう。しかし、パラメータの最適化のことを考えれば、できれば可能な限り少ないパラメータで変分フォームを構築したいですよね。

ここでは、まず$n=1$の場合を考えます。$U3$ゲートは3つのパラメータ$\theta$、$\phi$、$\lambda$を使って以下の変換を表現します:

\begin{align}
    U3(\theta, \phi, \lambda) = \begin{pmatrix}\cos\frac{\theta}{2} & -e^{i\lambda}\sin\frac{\theta}{2} \\ e^{i\phi}\sin\frac{\theta}{2} & e^{i\lambda + i\phi}\cos\frac{\theta}{2} \end{pmatrix}
\end{align}

グローバル位相を除いて、3つのパラメータを適切に設定して実装すれば任意の単一量子ビットの変換が行えます。この**ユニバーサル**な変分フォームは３つしかパラメータがないため効率的に最適化できるという特徴があります。ただ強調すべき点は、任意の状態を生成できるということは、この変分フォームが生成する状態は$H$の期待値を計算する上で必要になる状態に限定されないということです。つまり、この性質が意味するのは、最小の期待値が求まるかどうかは古典計算の最適化の能力だけに依存するということです。

### パラメータの最適化<a id='optimization'></a>
パラメータ化された変分フォームを選択したら、ターゲットとなるハミルトニアンの期待値を最小化するように、変分法に従ってパラメータを最適化する必要があります。パラメータの最適化のプロセスには様々な課題があります。例えば、量子ハードウェアには様々なタイプのノイズがあるため、その状態でエネルギーを測定しても正しい答えが返ってくるという保証はありません。そのために、パラメータの最適化に使う目的関数の評価が実際のエネルギーの値からずれてしまい、正しくパラメータの更新ができない可能性があります。また、最適化の手法（**オプティマイザー**）によっては、パラメータの数に依って目的関数を評価する回数が増えることがあり、さらにノイズの影響を受けやすくなります。つまり、アプリケーションの要求を考慮しながら、オプティマイザーの選択にも気を配る必要があります。

最も一般的な最適化手法は、エネルギーの減少が極大になるような方向に各パラメータを更新する**勾配降下法**です。各パラメータごとに勾配を計算するため、最適化すべきパラメータの数に応じて目的関数を評価する回数は増えます。また、この性質から探索空間の中で局所的な最適値を素早く発見することは可能ですが、逆に探索が局所的な最小値に留まってしまうことがあります。勾配降下法は直感的で理解しやすい最適化の手法ですが、少なくとも現在のNISQ量子コンピュータでは精度良く実行するのは難しいと考えられていて、現状ではあまり推奨されてはいません。

ノイズのある量子コンピュータで目的関数を最適化する適切なオプティマイザーとして、*Simultaneous Perturbation Stochastic Approximation*（**SPSA**）があります。SPSAは２回の測定だけで目的関数の勾配を近似できるという特徴があります。勾配降下法では各パラメータを独立に変化させるのに対して、SPSAでは全てのパラメータを同時にランダムに変化させます。以上のことから、現在のところVQEを利用する場合のオプティマイザーとしてはSPSAが推奨されているようです。

ノイズがない量子コンピュータで目的関数を評価する場合（例えば状態ベクトルシミュレータで実行する場合など）は、Pythonの[SciPy](https://www.scipy.org/scipylib/index.html)パッケージで提供されているオプティマイザーなど様々な選択肢があります。この実習では、Qiskit Aquaでサポートされているオプティマイザーの中で、特に*Constrained Optimization by Linear Approximation*（**COBYLA**）と呼ばれるオプティマイザーも使用します。COBYLAは目的関数の評価を1回しか実行しない（つまり評価の回数がパラメータの数に依存しない）ため、ノイズがない状態でかつ評価の回数を少なくしたい場合にはCOBYLAの利用が推奨されているようです。いづれにしろ、どのオプティマイザーがベストかはVQEアルゴリズムの実装形式や実行環境によって変わるため、ある程度経験によって決める必要があると考えられます。

### 変分フォームを使った実例<a id='example'></a>
ではここで、単一量子ビットの変分フォームを利用してパラメータ最適化の例を実行してみましょう。例として、ランダムな確率分布のベクトル$\vec{x}$を入力として与えた時、出力の確率分布が$\vec{x}$に近くなるように単一量子ビットの変分フォームを決定するという問題を考えます（2つの確率分布ベクトルの近さはL1-距離によって定義します）。

最初に、pythonでランダムな確率分布のベクトルを作成します。

In [1]:
import numpy as np
np.random.seed(999999)

target_distr = np.random.rand(2)
target_distr /= sum(target_distr)

次に、単一の$U_3$変分フォームの3つのパラメーターを引数として受け取り、対応する量子回路を返す関数を定義します。

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

変分フォームのパラメータのリストを入力とし、パラメータに対応したコストを計算する目的関数を定義します。アルゴリズムを実行するバックエンドとして、**QASMシミュレータ**を使用します。

In [3]:
from qiskit import Aer, execute
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(0)
    return output_distr

def objective_function(params):
    qc = get_var_form(params)
    result = execute(qc, backend, shots=NUM_SHOTS).result()
    output_distr = get_probability_distribution(result.get_counts(qc))
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost

最後にCOBYLAオプティマイザーのインスタンスを作成し、アルゴリズムを実行します。出力される確率分布は実行の度に異なり、ターゲットの確率分布と完全には同じにならないことに注意してください。出力の精度は量子計算の回数（ショット数＝NUM_SHOTS）に依存するので、ショット数を増減させた時の一致具合を確認してみてください。

In [4]:
from qiskit.aqua.components.optimizers import COBYLA

optimizer = COBYLA(maxiter=500, tol=0.0001)

params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

qc = get_var_form(ret[0])
counts = execute(qc, backend, shots=NUM_SHOTS).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (L1-Distance):", ret[1])
print("Parameters Found:", ret[0])

Target Distribution: [0.51357006 0.48642994]
Obtained Distribution: [0.5353, 0.4647]
Output Error (Manhattan Distance): 0.020459881261160884
Parameters Found: [ 1.52294877 -0.07797282  0.65499835]


## 高エネルギー実験への応用<a id='implementation'></a>
このセクションでは、高エネルギー実験へのVQEの応用を考えます。特に、高エネルギー粒子の衝突で発生する**荷電粒子の飛跡を再構成する**こと（**Tracking**と呼ばれます）への応用を考えます。
データとしては、[TrackML Particle Trackingチャレンジ](https://www.kaggle.com/c/trackml-particle-identification)で提供されたオープンデータを活用します。

まず、必要なライブラリを最初にインポートします。


In [5]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import TwoLocal
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver, NumPyEigensolver
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.aqua import QuantumInstance

### 量子演算の定義<a id='operators'></a>

次に、VQEでTrackingを実行するのに必要な量子演算を定義します。ここではシンプルな**1次元イジング模型**のハミルトニアンを考え、その最小固有値を求める問題としてTrackingを実行します。

In [6]:
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator

def get_ising_qubitops(J_ij,h_i,penalty=1e6):
    """Generate Hamiltonian for Ising model.

    Args:
        J_ij, h_ij : Coefficients of the couplings and linear terms
        penalty (float) : Penalty coefficient for the constraints

    Returns:
        operator.Operator, float: operator for the Hamiltonian and a
        constant shift for the obj function.
    """

    num_qubits = len(h_i)
    print("Number of qubits (selected triplets) =",num_qubits)
    pauli_list = []
    shift = 0

    zero = np.zeros(num_qubits, dtype=np.bool)
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i >= j:
                continue
            if J_ij[i][j] == 0:
                continue
            shift += J_ij[i][j]
            vp = np.zeros(num_qubits, dtype=np.bool)
            vp[i] = True
            vp[j] = True
            pauli_list.append([J_ij[i][j], Pauli(vp, zero)])

    zero = np.zeros(num_qubits, dtype=np.bool)
    for i in range(num_qubits):
        shift += h_i[i]
        vp = np.zeros(num_qubits, dtype=np.bool)
        vp[i] = True
        pauli_list.append([h_i[i], Pauli(vp, zero)])

    return WeightedPauliOperator(paulis=pauli_list), shift

$J_{ij}$は$i$と$j$番目の量子ビット間の相互作用の強さを表す係数で、$h_i$は$i$番目量子ビットに掛かる磁場の強さを表します。これらの係数をどのようにして決めるのかは、以下で説明します。量子演算としては、$J_{ij}$や$h_i$が係数として掛かったパウリ$Z$演算子(to check)を使って記述しています。

次に計算に用いるファイルを読み込んで、そこから$J_{ij}$と$h_i$を決定します。

In [7]:
file_r = 'data_files/QUBO_05pct_input.txt'
from ast import literal_eval
with open(file_r, "r") as f:
    line = f.read()
    Q = literal_eval(line)
print("Q size =",len(Q))


n_qubits = 100

nvar = 0
key_i = []
b_ij = np.zeros((n_qubits,n_qubits))
for (k1, k2), v in Q.items():
    if k1 == k2:
        b_ij[nvar][nvar] = v
        key_i.append(k1)
        nvar += 1

for (k1, k2), v in Q.items():
    if k1 != k2:
        for i in range(nvar):
            for j in range(nvar):
                if k1 == key_i[i] and k2 == key_i[j]:
                    if i < j:
                        b_ij[i][j] = v
                    else:
                        b_ij[j][i] = v

この実習では、**QUBO**（*Quadratic Unconstrained Binary Optimization*、2次制約無し2値最適化）と呼ばれる問題として与えられたモデルをVQEで解くことを考えます。QUBOはバイナリー値（0か1）で定義されますが、シンプルな変換でパウリ$Z$演算子の固有値（-1か1）としても表現できるため、上記の量子演算で同等の問題を扱うことができます。このQUBOの係数$b_{ij}$を求める部分は省略しますが、実際はいくつかの前処理を行って求めることになります。

### QUBO問題の導入<a id='qubo'></a>

説明を追加???

パウリ$Z$演算子の固有値を使って問題を解くため、以下では係数の変換を行います。

In [8]:
J_ij = np.zeros((nvar,nvar))
for i in range(nvar):
    for j in range(nvar):
        if i >= j:
            continue
        J_ij[i][j] = b_ij[i][j]
        if J_ij[i][j] == 0:
            continue

h_i = np.zeros(nvar)
for i in range(nvar):
    bias = 0
    for k in range(n_qubits):
        bias += b_ij[i][k]+b_ij[k][i]
    bias *= -1
    h_i[i] = bias
    if h_i[i] == 0:
        continue

### VQEによる近似解の探索<a id='vqe_tracking'></a>

ここまでは準備段階で、ここからVQEを使ってエネルギーの最小固有値（の近似解）を求めていくことになります。ただその前に、このハミルトニアンの行列を対角化して厳密にエネルギーの最小固有値とその固有ベクトルを求めた場合の答えを出してみましょう。

In [9]:
# === Solving Ising problem =====================
qubitOp, offset = get_ising_qubitops(J_ij,h_i)
print("")
print("total number of qubits = ",qubitOp.num_qubits)

# Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
ee = NumPyMinimumEigensolver(qubitOp)
result = ee.run()

print('Eigensolver: objective =', result.eigenvalue.real+offset)
x = sample_most_likely(result.eigenstate)
print('Eigensolver: x =',x)

samples_eigen = {}
for i in range(nvar):
    samples_eigen[key_i[i]] = x[i]

xのリストで1になっている量子ビットが最小エネルギーに対応するものとして選ばれているのが分かります。

次に、同じハミルトニアンモデルをVQEに実装して、最小エネルギーを求めてみます。オプティマイザーとしてはSPSAあるいはCOBYLAを使っています。オプティマイザーの繰り返し回数に依りますが、上で求めた厳密解と（ほぼ）同じ結果が得られることが分かると思います。

In [10]:
# --- run optimization with VQE
seed = 10598
spsa = SPSA(maxiter=300)
cobyla = COBYLA(maxiter=500)
two = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', 'linear', reps=1)
print(two)

#backend = Aer.get_backend('statevector_simulator')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend=backend, shots=1024, seed_simulator=seed)
vqe = VQE(qubitOp, two, spsa)
#vqe = VQE(qubitOp, two, cobyla)
result = vqe.run(quantum_instance)

print('')
#print(result['optimal_parameters'])
print('VQE: objective =', result.eigenvalue.real+offset)
x = sample_most_likely(result.eigenstate)
print('VQE x =',x)

samples_vqe = {}
for i in range(nvar):
    samples_vqe[key_i[i]] = x[i]

### おまけ<a id='omake'></a>

Trackingがうまく行っても、この答えでは面白くないですよね。正しく飛跡が見つかったかどうか目で確認するため、以下のコードを走らせてみましょう。

このコードは、QUBOを定義する時に使った検出器のヒット位置をビーム軸に垂直な平面でプロットして、どのヒットが選ばれたかを分かりやすく可視化したものです。緑の線が実際に見つかった飛跡で、青の線を含めたものが全体の飛跡の候補です。この実習では限られた数の量子ビットしか使っていないため、大部分の飛跡は見つけられていませんが、緑の線から計算に使った3点ヒットからは正しく飛跡が見つかっていることが分かると思います。

In [11]:
from hepqpr.qallse import *
input_path = './data_files/event000001000-hits.csv'
dw = DataWrapper.from_path(input_path)

# get the results
#all_doublets = Qallse.process_sample(samples_eigen)
all_doublets = Qallse.process_sample(samples_vqe)

final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)
#print("all_doublets =",all_doublets)
#print("final_tracks =",final_tracks)
#print("final_doublets =",final_doublets)

p, r, ms = dw.compute_score(final_doublets)
trackml_score = dw.compute_trackml_score(final_tracks)

print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(ms)}')
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

from hepqpr.qallse.plotting import iplot_results, iplot_results_tracks
dims = ['x', 'y']
_, missings, _ = diff_rows(final_doublets, dw.get_real_doublets())
dout = 'plot-ising_found_tracks.html'
iplot_results(dw, final_doublets, missings, dims=dims, filename=dout)


## 参考文献<a id='references'></a>
1. Nielsen, Michael A and Chuang, Isaac L, "Quantum Computation and Quantum Information", Cambridge University Pres, 2000.
2. Peruzzo, Alberto, et al., "A variational eigenvalue solver on a photonic quantum processor", [Nature commun. 5, 4213 (2014)](https://www.nature.com/articles/ncomms5213).